In [43]:
from mlxModel import MLXChatModel

In [44]:
# mlx_llm = MLXChatModel(mlx_path = "/Users/austinlackey/.cache/huggingface/hub/models--mlx-community--llama2-13b-qnt4bit/snapshots/02e61fe8846d8b9c94d658227410ab98af56b063")
# mlx_llm = MLXChatModel(mlx_path = "/Users/austinlackey/.cache/huggingface/hub/models--mlx-community--Meta-Llama-3-8B-4bit/snapshots/d6641889c828724531aef71cb2d14efb7c9cfbd5")
mlx_llm = MLXChatModel(mlx_path = "/Users/austinlackey/.cache/huggingface/hub/models--mlx-community--Meta-Llama-3-8B-Instruct-4bit/snapshots/c38b3b1f03cce0ce0ccd235e5c97b0d3d255e651")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [53]:
from langchain.tools import BaseTool
from typing import Union
import pandas as pd

class FieldCodeTool(BaseTool):
    name = "Find the field code for a given field name"
    description = "use this tool when you need to find the field code for a given field name"

    def _run(self, field_name: str):
        df = pd.read_excel("Data/Test_field.xlsx")
        # return cell if field name contains field name
        field_code = df[df["FIELD NAME"].str.contains(field_name, case=False)]["FIELD CODE"]
        # If series is [] then return None
        if field_code.empty:
            return "<<NFC>>"
        return field_code.values[0]
    
    def _arun(self, field_name: str):
        raise NotImplementedError("This tool does not support async")

class FieldNameTool(BaseTool):
    name = "Get a list of field names"
    description = "use this tool when you need to get a list of field names"

    def _run(self, field_name: str):
        df = pd.read_excel("Data/Test_field.xlsx")
        return df["FIELD NAME"].tolist()
    
    def _arun(self):
        raise NotImplementedError("This tool does not support async")

print(FieldCodeTool().run("NEMO BRIDGE"))
print(FieldNameTool().run(

F1


TypeError: BaseTool.run() missing 1 required positional argument: 'tool_input'

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
)

In [50]:
from langchain.agents import initialize_agent

tools = [FieldCodeTool()]

# initialize agent with tools
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=mlx_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [51]:
sys_msg = """Assistant is a large language model trained by Meta.

Assistant is designed to be able to assist with a wide range of tasks that surrounds The National Parks. Assistant is meant to summarize monthly comments regarding Park visitation, entrance/trail closures, and other important information. Assistant should find entrances/trails/areas of the park that were mentioned in the comment and provide a short, few word summary of what happened at that location.

Comments will mention areas of the park that received increases, decreases or closures in visitation, and it is the Assistant's job to summarize what happened at each location mentioned in the comment.

These locations, or areas will be referred to as Field Names, and each Field Name has a corresponding Field Code. Field Codes must be searched for using the Field Code Tool.

If <<NFC>> is returned, it means that the Field Name was not found in the Field Code Tool, that is okay, just return <<NFC>> as the Field Code.
Although, Assistant does not have any information on what Field Codes correspond to which park entrance/trail/area, so it will always refer to its trusty tools and absolutely does NOT try to guess answers by itself.

"""

new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

In [52]:
agent("Nemo Bridge is malfunctioning.")



> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'chat_history'}